# Building a Spam Filter with Naive Bayes

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. We will be aiming for an accuracy of 80%, meaning that _at least_ 4 of every 5 spam messages will be marked as such.

The dataset we'll be working with is a collection of 5,572 human labeled SMS messages. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo. The dataset can be downloaded from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). To read more about the data collection process, click [here](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).

In [1]:
import pandas as pd
import numpy as np

sms = pd.read_csv('SMSSpamCollection', sep = '\t', names = ['Label', 'SMS'])
print('The dataset has {} rows and {} columns'.format(sms.shape[0], sms.shape[1]))

The dataset has 5572 rows and 2 columns


In [2]:
spam_per = (sum(sms['Label'] == 'spam'))/sms.shape[0] * 100
spam_per
print('{0:.2f} % of the messages are spam'.format(spam_per))

13.41 % of the messages are spam


In [3]:
# Shuffle the rows of the dataframe and split into train and test sets
sms_shuffled = sms.sample(frac = 1, random_state = 1)
train = sms_shuffled[:4458].reset_index(drop = True)
test = sms_shuffled[4458:].reset_index(drop = True)

In [4]:
# Find the relevant amounts of spam and ham in the two subsets
train_spam = (sum(train['Label'] == 'spam'))/train.shape[0] * 100
test_spam = (sum(test['Label'] == 'spam'))/test.shape[0] * 100

print('The training set is {0:.2f} % spam and the test set is {1:.2f} % spam'.format(train_spam, test_spam))

The training set is 13.46 % spam and the test set is 13.20 % spam


We can see that the relative amount of spam in both datasets is rather close to the amount of spam in the overall data set. Next we will remove all the punctuation from our messages and convert all the words to lowercase. This will allow us to more easily construct a matrix with all the unique values of words in the message repository.

We perform some minor string formatting to normalize all the words, allowing us to not have multiple entries for words with mismatched capitalization or various punctuation.

In [5]:
train['SMS'] = train['SMS'].str.replace('\W', ' ')
train['SMS'] = train['SMS'].str.lower()
train.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


Next we will continue formatting our data for our analysis. Naive Bayes analysis requires we have the probability of each word occurring in spam and non-spam (ham) messages. To achieve this, we need to create a dictionary that contains the vocabulary used in ALL the messages and how many times they occur in that group of messages.  

In [6]:
train['SMS'] = train['SMS'].str.split()
vocabulary = []
for sms in train['SMS']:
    for word in sms:
        vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [7]:
len(vocabulary)

7783

Finally, we will transform our training dataset into the final format we wish to use.

In [8]:
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [9]:
words = pd.DataFrame(word_counts_per_sms)
words.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [10]:
train_new = pd.concat([train, words], axis = 1)
train_new.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


### Calculating Constants

Now that we've cleaned and formatted the training set into the desired structure, we can begin creating our spam filter. The Naive Bayes algorithm will compare the probabilities of the message being spam or ham and use the higher of the two to classify the message. The equation used is:

$P(Spam|w_1, w_2, ..., w_n)$  $\alpha$  $P(spam) * \prod P(w_i|Spam)$ where i = 1, 2, ..., n

To calculate the probabilities P(wi|Spam) and P(wi|Ham) we use the following equations:

$ P(w_i|Spam) = \frac{N_{wi}|Spam  +  \alpha}{N_{Spam} + \alpha* N_{vocabulary}} $ and  $ P(w_i|Ham) = \frac{N_{wi}|Ham  +  \alpha}{N_{Ham} + \alpha* N_{vocabulary}} $

Some of these terms will remain constant from message to message. We can calculate the value of these terms once and then reuse them for all subsequent computations. These values are:
  * P(spam) and P(Ham)
  * $N_{Spam}, N_{Ham}, N_{Vocabulary}$
  
We also implement Laplace smoothing with $\alpha$ = 1

In [11]:
# Calculate P(Spam), P(Ham), N_spam, N_ham, and N_vocabulary
p_spam = sum(train_new['Label'] == 'spam')/train_new.shape[0]
p_ham = 1 - p_spam

print(p_spam, p_ham)

spam_train = train_new[train_new['Label'] == 'spam']
n_spam = len(spam_train['SMS'].sum())

ham_train = train_new[train_new['Label'] == 'ham']
n_ham = len(ham_train['SMS'].sum())

n_vocabulary = len(vocabulary)
alpha = 1

0.13458950201884254 0.8654104979811574
15190
57237


### Calculating Parameters

Next we must calculate the conditional probabilities given by the formulas in the previous section.

In [17]:
# Calculate the conditional probabilities of every word given the word is or 
# isn't spam
p_word_given_spam = {unique_word:0 for unique_word in vocabulary}
p_word_given_ham = {unique_word:0 for unique_word in vocabulary}
    
def p_cond(n_w, alpha, n_label, n_vocab):
    num = n_w + alpha
    denom = n_label + alpha * n_vocab
    return num/denom

In [20]:
spam_train = train_new[train_new['Label'] == 'spam']
ham_train = train_new[train_new['Label'] == 'ham']

for word in vocabulary:
    n_w_spam = spam_train[word].sum()
    p_w_spam = p_cond(n_w_spam, alpha, n_spam, n_vocabulary)
    p_word_given_spam[word] = p_w_spam
    
    n_w_ham = ham_train[word].sum()
    p_w_ham = p_cond(n_w_ham, alpha, n_ham, n_vocabulary)
    p_word_given_ham[word] = p_w_ham

p_word_given_spam

{'kiosk': 0.0001305880816610804,
 'diwali': 4.3529360553693465e-05,
 'notified': 8.705872110738693e-05,
 'brings': 8.705872110738693e-05,
 'pest': 4.3529360553693465e-05,
 'limping': 4.3529360553693465e-05,
 'agents': 4.3529360553693465e-05,
 'whassup': 4.3529360553693465e-05,
 'high': 0.0001305880816610804,
 'degree': 4.3529360553693465e-05,
 'pending': 4.3529360553693465e-05,
 'bfore': 4.3529360553693465e-05,
 'hudgi': 4.3529360553693465e-05,
 'mummy': 4.3529360553693465e-05,
 'sea': 4.3529360553693465e-05,
 'machi': 4.3529360553693465e-05,
 'dogs': 8.705872110738693e-05,
 'honesty': 4.3529360553693465e-05,
 'greeting': 4.3529360553693465e-05,
 'fromwrk': 4.3529360553693465e-05,
 'hell': 4.3529360553693465e-05,
 'respond': 8.705872110738693e-05,
 'notifications': 8.705872110738693e-05,
 'representative': 0.00030470552387585427,
 'cricket': 4.3529360553693465e-05,
 'boyfriend': 4.3529360553693465e-05,
 'curtsey': 4.3529360553693465e-05,
 'wouldn': 4.3529360553693465e-05,
 'emerging': 

### Classifying a New Message

Now that we have all the pieces of the equation, we can build our spam filter. The filter will be a function that:

  * Takes in a new message as input
  * Calculates the likelihood of it being Spam or Ham
  * Compares these likelihoods and assigns a label based on which is greater

In [23]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    '''    
    This is where we calculate:

    p_spam_given_message = ?
    p_ham_given_message = ?
    '''    
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_word_given_spam:
            p_spam_given_message *= p_word_given_spam[word]
        
        if word in p_word_given_ham:
            p_ham_given_message *= p_word_given_ham[word]
            

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [24]:
message1 = 'WINNER!! This is the secret code to unlock the money: C3421'
message2 = "Sounds good, Tom, then see u there"

classify(message1)
classify(message2)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam
P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


In [27]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    '''    
    This is where we calculate:

    p_spam_given_message = ?
    p_ham_given_message = ?
    '''    
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_word_given_spam:
            p_spam_given_message *= p_word_given_spam[word]
        
        if word in p_word_given_ham:
            p_ham_given_message *= p_word_given_ham[word]
            

    # print('P(Spam|message):', p_spam_given_message)
    # print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification this!'

### Testing Accuracy

Now that we've seen our filter in action, let's see how it performs on the test set separated at the start of the project.

In [31]:
test['predicted'] = test['SMS'].apply(classify)
correct = 0
total = test.shape[0]


for index, row in test.iterrows():
    if row['predicted'] == row['Label']:
        correct += 1
        
accuracy = correct/total
print(accuracy)

0.9874326750448833


# Conclusion

In this project we used the multinomial Naive Bayes algorithim to create a filter for identifying SMS messages as spam. We managed to create a filter that achieved an accuracy of 98.7% in correctly classifying the messages as either spam or ham.